In [3]:
import requests
import os

In [5]:
import pandas as pd

In [56]:
import pprint

In [64]:
import json

In [4]:
foursquare_api_key = os.environ['FOURSQUARE_STATS_API_KEY']

In [1]:
#%store -r lat_lon

In [108]:
lat_lon_init = pd.read_csv('../data/lat_lon.csv')
lat_lon = lat_lon_init.drop(columns=lat_lon_init.columns[0], axis=1)
lat_lon

,0,1
0,53.350230,-6.279696
1,53.357841,-6.251557
2,53.356307,-6.273717
3,53.349562,-6.278198
4,53.336021,-6.262980
...,...,...
109,53.336597,-6.248109
110,53.353742,-6.265301
111,53.344603,-6.263371
112,53.347692,-6.278214


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [87]:
def foursquare_request(latitude, longitude):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        # retrieve latitude and longitude from list of bike station location coordinates, and convert to string
        "ll": f"{latitude},{longitude}",
        "radius": 1000,
        "categories": "13000, 19028, 19043", # search for dining and drinking, bus stops, and tourist information and service
        "sort": "DISTANCE",
        "fields": "name,distance,popularity,rating,categories" 
    }

    headers = {
        "Accept": "application/json",
        "Authorization": f"{foursquare_api_key}"
    }

    foursquare_result = requests.request("GET", url, params=params, headers=headers)

    return foursquare_result.json()

# create empty list to contain a dataframe for each location's associated information
final_location_data_list = []

# iterate through bike station locations
for location in lat_lon:
    latitude = location[0]
    longitude = location[1]
    print(f'getting results for {latitude}, {longitude}')
    result = foursquare_request(latitude, longitude) # assign location json results to a variable
    data = result.get('results') # extract data located in the 'results' header of the json file
    foursquare_df = pd.DataFrame(data) # convert location results to a pandas dataframe
    foursquare_df['latitude'] = latitude
    foursquare_df['longitude'] = longitude 
    foursquare_df['category'] = foursquare_df['categories'].map(lambda x: x[0].get('name')) # extract only the first category name from a venue's category info
    foursquare_df = foursquare_df.drop('categories', axis = 1) # remove the 'categories' column which contains a list of dictionaries
    final_location_data_list.append(foursquare_df)

final_df = pd.concat(final_location_data_list, axis=0) # convert list of location dataframes to one big dataframe

final_df.head(10)

getting results for 53.35023, -6.279696
getting results for 53.357841, -6.251557
getting results for 53.356307, -6.273717
getting results for 53.349562, -6.278198
getting results for 53.336021, -6.26298
getting results for 53.359405, -6.276142
getting results for 53.33796, -6.24153
getting results for 53.343368, -6.27012
getting results for 53.334123, -6.265436
getting results for 53.344304, -6.250427
getting results for 53.338755, -6.262003
getting results for 53.347777, -6.244239
getting results for 53.336074, -6.252825
getting results for 53.330091, -6.268044
getting results for 53.350929, -6.265125
getting results for 53.341515, -6.256853
getting results for 53.348279, -6.254662
getting results for 53.35893, -6.280337
getting results for 53.344115, -6.237153
getting results for 53.343893, -6.280531
getting results for 53.347477, -6.28525
getting results for 53.339005, -6.300217
getting results for 53.344153, -6.233451
getting results for 53.334295, -6.258503
getting results for 53.

,distance,name,popularity,rating,latitude,longitude,category
0,37,V-Face,0.961758,NaN,53.35023,-6.279696,Vegan and Vegetarian Restaurant
1,131,The Cobblestone,0.989030,8.5,53.35023,-6.279696,Pub
2,165,Oscars Café Bar,0.981643,6.7,53.35023,-6.279696,Bar
3,177,Light House Cinema,0.987419,9.2,53.35023,-6.279696,Indie Movie Theater
4,235,Generator Hostel bar,0.973561,NaN,53.35023,-6.279696,Bar
5,285,JJ's Bar,0.995279,8.3,53.35023,-6.279696,Cocktail Bar
6,286,Token,0.974089,8.7,53.35023,-6.279696,Arcade
7,287,The Reserve Bar,0.988975,7.5,53.35023,-6.279696,Whisky Bar
8,291,Old Jameson Distillery,0.998806,8.9,53.35023,-6.279696,Tour Provider
9,297,Proper Order Coffee Co.,0.983281,8.1,53.35023,-6.279696,Café


In [88]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 0 to 9
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   distance    1140 non-null   int64  
 1   name        1140 non-null   object 
 2   popularity  1111 non-null   float64
 3   rating      776 non-null    float64
 4   latitude    1140 non-null   float64
 5   longitude   1140 non-null   float64
 6   category    1140 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 71.2+ KB


### Troubleshooting code snippets

run the foursquare request once, figure out where the category information is within the foursquare result structure.

In [58]:
url = "https://api.foursquare.com/v3/places/search"
params = {
    # retrieve latitude and longitude from list of bike station location coordinates, and convert to string
    "ll": "53.35023,-6.279696",
    "radius": 1000,
    "categories": "13000, 19028, 19043", # search for dining and drinking, bus stops, and tourist information and service
    "sort": "DISTANCE",
    "fields": "name,distance,popularity,rating,categories" 
}

headers = {
    "Accept": "application/json",
    "Authorization": f"{foursquare_api_key}"
}

foursquare_result = requests.request("GET", url, params=params, headers=headers)

pprint.pprint(foursquare_result.json())

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 53.35023,
                                                  'longitude': -6.279696},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
                                       'suffix': '.png'},
                              'id': 13377,
                              'name': 'Vegan and Vegetarian Restaurant'}],
              'distance': 37,
              'name': 'V-Face',
              'popularity': 0.9622954097517781},
             {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
                                       'suffix': '.png'},
                              'id': 13018,
                              'name': 'Pub'}],
              'distance': 131,
              'name': 'The Cobblestone',
              'popularity': 0.9895169813476534,
              'rating': 8.

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Points of interest:
- dining and drinking (13000)
    - coffee shops (13032)
    - brewery (13029)
    - bars (13003)
    - restaurant (13065)
- bus stop (19043)
- tourist information and service (19028)

POIs that were considered but not included:
- public bathroom (12092)
- historic and protected site (16020)
- souvenir store (17116)
- gift shops (17089)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating